In [1]:
import sympy as sp
from sympy import symbols, pprint
from sympy import sin, cos, asin, acos, pi
from sympy import Matrix, simplify, Function, diff, Derivative, nsimplify, lambdify

import numpy as np

# from IPython import display # for the animation
import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.pyplot as plt
plt.ion()

In [ ]:
def T(theta, x, y):
    return Matrix([[cos(theta), -sin(theta), x], 
                   [sin(theta), cos(theta), y],
                   [0, 0, 1]])

def sym_to_np(T):
    return np.array(T).astype(np.float64)

In [11]:
def draw_robot(ordered_list_of_transformations):

    ax_lw = 3
    link_lw = 1

    # homogeneous unit vectors 
    x = np.array([[1, 0, 1]]).T 
    y = np.array([[0, 1, 1]]).T
    origin = np.array([[0, 0, 1]]).T

    prev_origin = np.array([[0, 0, 1]]).T    
    current_transformation = np.eye(3)

    plt.clf()
    ax = plt.gca()  # get current axis

    # now we plot
    plt.plot([prev_origin[0][0]], [prev_origin[1][0]], '-ko',linewidth=link_lw)
    plt.plot([prev_origin[0][0], x[0][0]], [prev_origin[1][0], x[1][0]], '-ro', linewidth=ax_lw)
    plt.plot([prev_origin[0][0], y[0][0]], [prev_origin[1][0], y[1][0]], '-go', linewidth=ax_lw)


    # loop the transforms in order of T_01, T_12, ... , T_N-1N
    for k, transform in enumerate(ordered_list_of_transformations):

        # update the transformation
        current_transformation = current_transformation @ transform
        new_origin = current_transformation @ origin
        new_x = current_transformation @ x
        new_y = current_transformation @ y

        # now we plot
        plt.plot([prev_origin[0][0], new_origin[0][0]], 
                 [prev_origin[1][0], new_origin[1][0]], 
                 '-ko',linewidth=link_lw)
        
        plt.plot([new_origin[0][0], new_x[0][0]], [new_origin[1][0], new_x[1][0]], '-r', linewidth=ax_lw)
        plt.plot([new_origin[0][0], new_y[0][0]], [new_origin[1][0], new_y[1][0]], '-g', linewidth=ax_lw)


#         ax.annotate('{'+str(k+1)+'}', xy=new_origin[0:2][0], xytext=new_origin[0:2][0])

        # lastly update the current_origin to the new_origin
        prev_origin = new_origin

    # lastly set the axes to be equal and appropriate
    ax.set_aspect('equal')
    ax.set_xlabel('x')
    ax.set_ylabel('y')    

## Problem 1

In [14]:
(t, theta_1, theta_2, theta_3, l_1, l_2, l_3, s) = symbols('t, theta_1 theta_2 theta_3 l_1 l_2 l_3 s', real = True)

theta_1 = Function('theta_1', real=True)(t)
theta_2 = Function('theta_2', real=True)(t)
theta_3 = Function('theta_3', real=True)(t)

In [15]:
T_01 = T(theta_1, 0, 0)
T_12 = T(theta_2, l_1, 0)
T_23 = T(theta_3, l_2, 0)

In [16]:
FK1 = (T_01 * Matrix([[s], [0], [1]]))[:-1,0]
FK2 = simplify((T_01 * T_12 * Matrix([[s], [0], [1]]))[:-1,0])
FK3 = simplify((T_01 * T_12 * T_23 * Matrix([[s], [0], [1]]))[:-1,0])

In [17]:
FK1

Matrix([
[s*cos(theta_1(t))],
[s*sin(theta_1(t))]])

In [18]:
FK2

Matrix([
[l_1*cos(theta_1(t)) + s*cos(theta_1(t) + theta_2(t))],
[l_1*sin(theta_1(t)) + s*sin(theta_1(t) + theta_2(t))]])

In [19]:
FK3

Matrix([
[l_1*cos(theta_1(t)) + l_2*cos(theta_1(t) + theta_2(t)) + s*cos(theta_1(t) + theta_2(t) + theta_3(t))],
[l_1*sin(theta_1(t)) + l_2*sin(theta_1(t) + theta_2(t)) + s*sin(theta_1(t) + theta_2(t) + theta_3(t))]])

In [20]:
J1 = FK1.jacobian([theta_1, theta_2, theta_3])
J2 = simplify(FK2.jacobian([theta_1, theta_2, theta_3]))
J3 = simplify(FK3.jacobian([theta_1, theta_2, theta_3]))

In [21]:
J1

Matrix([
[-s*sin(theta_1(t)), 0, 0],
[ s*cos(theta_1(t)), 0, 0]])

In [22]:
J2

Matrix([
[-l_1*sin(theta_1(t)) - s*sin(theta_1(t) + theta_2(t)), -s*sin(theta_1(t) + theta_2(t)), 0],
[ l_1*cos(theta_1(t)) + s*cos(theta_1(t) + theta_2(t)),  s*cos(theta_1(t) + theta_2(t)), 0]])

In [23]:
J3

Matrix([
[-l_1*sin(theta_1(t)) - l_2*sin(theta_1(t) + theta_2(t)) - s*sin(theta_1(t) + theta_2(t) + theta_3(t)), -l_2*sin(theta_1(t) + theta_2(t)) - s*sin(theta_1(t) + theta_2(t) + theta_3(t)), -s*sin(theta_1(t) + theta_2(t) + theta_3(t))],
[ l_1*cos(theta_1(t)) + l_2*cos(theta_1(t) + theta_2(t)) + s*cos(theta_1(t) + theta_2(t) + theta_3(t)),  l_2*cos(theta_1(t) + theta_2(t)) + s*cos(theta_1(t) + theta_2(t) + theta_3(t)),  s*cos(theta_1(t) + theta_2(t) + theta_3(t))]])

In [24]:
# Lambdify symbolic expressions
J1_func = lambdify((s, l_1, l_2, l_3, theta_1, theta_2, theta_3), J1)
J2_func = lambdify((s, l_1, l_2, l_3, theta_1, theta_2, theta_3), J2)
J3_func = lambdify((s, l_1, l_2, l_3, theta_1, theta_2, theta_3), J3)

FK1_func = lambdify((s, l_1, l_2, l_3, theta_1, theta_2, theta_3), FK1)
FK2_func = lambdify((s, l_1, l_2, l_3, theta_1, theta_2, theta_3), FK2)
FK3_func = lambdify((s, l_1, l_2, l_3, theta_1, theta_2, theta_3), FK3)

In [55]:
link1 = 1  
link2 = 1
link3 = 1

theta1 = 45 * np.pi/180 
theta2 = -60 * np.pi/180
theta3 = -30 * np.pi/180

theta1_time = np.linspace(0, theta1, 50)
theta2_time = np.linspace(0, theta2, 50)
theta3_time = np.linspace(0, theta3, 50)

dtheta1_time = np.gradient(np.linspace(0, theta1, 50))
dtheta2_time = np.gradient(np.linspace(0, theta2, 50))
dtheta3_time = np.gradient(np.linspace(0, theta3, 50))

fig = plt.figure(1,figsize = (8,8))

# Main simulation loop
for theta1_current, theta2_current, theta3_current, v1, v2, v3 in zip(theta1_time, theta2_time, theta3_time, dtheta1_time, dtheta2_time, dtheta3_time):
    # Make transformation matrices
    T_01_real = sym_to_np(T_01.subs([(theta_1, theta1_current)]))
    T_12_real = sym_to_np(T_12.subs([(theta_2, theta2_current), (l_1, link1)]))
    T_23_real = sym_to_np(T_23.subs([(theta_3, theta3_current), (l_2, link2)]))
    T_34_real = sym_to_np(T(0, link3, 0))
    
    draw_robot([T_01_real, T_12_real, T_23_real, T_34_real])

    # F=-v
    for ss in np.linspace(0.1, 1, 10):
        # Evaluate Jacobians and FK using lambdified functions
        vel1 = J1_func(ss*link1, link1, link2, link3, theta1_current, theta2_current, theta3_current) @ np.array([v1, v2, v3])
        pos1 = FK1_func(ss*link1, link1, link2, link3, theta1_current, theta2_current, theta3_current)
        
        plt.quiver(float(pos1[0]), float(pos1[1]), float(-vel1[0]), float(-vel1[1]))
        
        vel2 = J2_func(ss*link2, link1, link2, link3, theta1_current, theta2_current, theta3_current) @ np.array([v1, v2, v3])
        pos2 = FK2_func(ss*link2, link1, link2, link3, theta1_current, theta2_current, theta3_current)
        
        plt.quiver(float(pos2[0]), float(pos2[1]), float(-vel2[0]), float(-vel2[1]))
        
        vel3 = J3_func(ss*link3, link1, link2, link3, theta1_current, theta2_current, theta3_current) @ np.array([v1, v2, v3])
        pos3 = FK3_func(ss*link3, link1, link2, link3, theta1_current, theta2_current, theta3_current)
        
        plt.quiver(float(pos3[0]), float(pos3[1]), float(-vel3[0]), float(-vel3[1]))

    sc = 25
    plt.axis([-1, 4, -2, 3])
    plt.draw()
    plt.pause(0.01)


C:\Users\78469\AppData\Local\Temp\ipykernel_37824\2527193379.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  plt.quiver(float(pos1[0]), float(pos1[1]), float(-vel1[0]), float(-vel1[1]))
C:\Users\78469\AppData\Local\Temp\ipykernel_37824\2527193379.py:40: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  plt.quiver(float(pos2[0]), float(pos2[1]), float(-vel2[0]), float(-vel2[1]))
C:\Users\78469\AppData\Local\Temp\ipykernel_37824\2527193379.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. 

In [71]:
theta1_time

array([0.        , 0.01602853, 0.03205707, 0.0480856 , 0.06411414,
       0.08014267, 0.0961712 , 0.11219974, 0.12822827, 0.14425681,
       0.16028534, 0.17631387, 0.19234241, 0.20837094, 0.22439948,
       0.24042801, 0.25645654, 0.27248508, 0.28851361, 0.30454214,
       0.32057068, 0.33659921, 0.35262775, 0.36865628, 0.38468481,
       0.40071335, 0.41674188, 0.43277042, 0.44879895, 0.46482748,
       0.48085602, 0.49688455, 0.51291309, 0.52894162, 0.54497015,
       0.56099869, 0.57702722, 0.59305576, 0.60908429, 0.62511282,
       0.64114136, 0.65716989, 0.67319843, 0.68922696, 0.70525549,
       0.72128403, 0.73731256, 0.7533411 , 0.76936963, 0.78539816])

In [61]:
torque1 = J1.T * -vel1
torque2 = J2.T * -vel2
torque3 = J3.T * -vel3
torque = [torque1, torque2, torque3]
torque

[array([[-0.0113338850463224*s*sin(theta_1(t)),
         -0.0113338850463224*s*cos(theta_1(t))],
        [0, 0],
        [0, 0]], dtype=object),
 array([[-0.0127167149964979*l_1*sin(theta_1(t)) - 0.0127167149964979*s*sin(theta_1(t) + theta_2(t)),
         -0.00617309341403948*l_1*cos(theta_1(t)) - 0.00617309341403948*s*cos(theta_1(t) + theta_2(t))],
        [-0.0127167149964979*s*sin(theta_1(t) + theta_2(t)),
         -0.00617309341403948*s*cos(theta_1(t) + theta_2(t))],
        [0, 0]], dtype=object),
 array([[-0.0240506000428202*l_1*sin(theta_1(t)) - 0.0240506000428202*l_2*sin(theta_1(t) + theta_2(t)) - 0.0240506000428202*s*sin(theta_1(t) + theta_2(t) + theta_3(t)),
         0.00516079163228284*l_1*cos(theta_1(t)) + 0.00516079163228284*l_2*cos(theta_1(t) + theta_2(t)) + 0.00516079163228284*s*cos(theta_1(t) + theta_2(t) + theta_3(t))],
        [-0.0240506000428202*l_2*sin(theta_1(t) + theta_2(t)) - 0.0240506000428202*s*sin(theta_1(t) + theta_2(t) + theta_3(t)),
         0.005160791632

In [ ]:
fig = plt.figure(1,figsize = (8,8))

# Main simulation loop
for theta1_current, theta2_current, theta3_current, v1, v2, v3 in zip(theta1_time, theta2_time, theta3_time, dtheta1_time, dtheta2_time, dtheta3_time):
    # Make transformation matrices
    T_01_real = sym_to_np(T_01.subs([(theta_1, theta1_current)]))
    T_12_real = sym_to_np(T_12.subs([(theta_2, theta2_current), (l_1, link1)]))
    T_23_real = sym_to_np(T_23.subs([(theta_3, theta3_current), (l_2, link2)]))
    T_34_real = sym_to_np(T(0, link3, 0))
    
    draw_robot([T_01_real, T_12_real, T_23_real, T_34_real])

    F1 = -J1_func(1, link1, link2, link3, theta1_current, theta2_current, theta3_current) @ np.array([v1, v2, v3])
    torque1 = J1_func(1, link1, link2, link3, theta1_current, theta2_current, theta3_current).T * F1
    
    # plt.quiver(float(torque1[0]), float(torque1[1]), float(F1[0]), float(F1[1]))
    
    F2 = -J2_func(1, link1, link2, link3, theta1_current, theta2_current, theta3_current) @ np.array([v1, v2, v3])
    torque2 = J2_func(1, link1, link2, link3, theta1_current, theta2_current, theta3_current).T * F2

    
    F3 = -J3_func(1, link1, link2, link3, theta1_current, theta2_current, theta3_current) @ np.array([v1, v2, v3])
    torque3 = J3_func(1, link1, link2, link3, theta1_current, theta2_current, theta3_current).T * F3


In [72]:
# two links one joint
FK1 = (T_01 * Matrix([[l_1], [0], [1]]))[:-1,0]
FK2 = simplify((T_01 * T_12 * Matrix([[l_2], [0], [1]]))[:-1,0])
J1 = FK1.jacobian([theta_1, theta_2])
J2 = simplify(FK2.jacobian([theta_1, theta_2]))

In [ ]:
# When the system starts and ends at the same angle, 
# the forces inside the system cancel each other out, resulting in a net force of zero.

## Problem 2

In [153]:
(t, theta_1, theta_2, theta_3, l) = symbols('t, theta_1 theta_2 theta_3 l', real = True)

theta_1 = Function('theta_1', real=True)(t)
theta_2 = Function('theta_2', real=True)(t)
theta_3 = Function('theta_3', real=True)(t)

In [154]:
T_01 = T(theta_1, 0, 0)
T_12 = T(theta_2, l, 0)
T_23 = T(theta_3, l, 0)

In [157]:
FK1 = (T_01 * Matrix([[l], [0], [1]]))[:-1,0]
FK2 = simplify((T_01 * T_12 * Matrix([[l], [0], [1]]))[:-1,0])
FK3 = simplify((T_01 * T_12 * T_23 * Matrix([[l], [0], [1]]))[:-1,0])
J1 = FK1.jacobian([theta_1, theta_2, theta_3])
J2 = simplify(FK2.jacobian([theta_1, theta_2, theta_3]))
J3 = simplify(FK3.jacobian([theta_1, theta_2, theta_3]))

In [158]:
FK1

Matrix([
[l*cos(theta_1(t))],
[l*sin(theta_1(t))]])

In [159]:
FK2

Matrix([
[l*(cos(theta_1(t) + theta_2(t)) + cos(theta_1(t)))],
[l*(sin(theta_1(t) + theta_2(t)) + sin(theta_1(t)))]])

In [160]:
FK3

Matrix([
[l*(cos(theta_1(t) + theta_2(t)) + cos(theta_1(t) + theta_2(t) + theta_3(t)) + cos(theta_1(t)))],
[l*(sin(theta_1(t) + theta_2(t)) + sin(theta_1(t) + theta_2(t) + theta_3(t)) + sin(theta_1(t)))]])

In [161]:
J1

Matrix([
[-l*sin(theta_1(t)), 0, 0],
[ l*cos(theta_1(t)), 0, 0]])

In [162]:
J2

Matrix([
[-l*(sin(theta_1(t) + theta_2(t)) + sin(theta_1(t))), -l*sin(theta_1(t) + theta_2(t)), 0],
[ l*(cos(theta_1(t) + theta_2(t)) + cos(theta_1(t))),  l*cos(theta_1(t) + theta_2(t)), 0]])

In [163]:
J3

Matrix([
[-l*(sin(theta_1(t) + theta_2(t)) + sin(theta_1(t) + theta_2(t) + theta_3(t)) + sin(theta_1(t))), -l*(sin(theta_1(t) + theta_2(t)) + sin(theta_1(t) + theta_2(t) + theta_3(t))), -l*sin(theta_1(t) + theta_2(t) + theta_3(t))],
[ l*(cos(theta_1(t) + theta_2(t)) + cos(theta_1(t) + theta_2(t) + theta_3(t)) + cos(theta_1(t))),  l*(cos(theta_1(t) + theta_2(t)) + cos(theta_1(t) + theta_2(t) + theta_3(t))),  l*cos(theta_1(t) + theta_2(t) + theta_3(t))]])

In [164]:
J = Matrix([[J1], [J2], [J3]])
J

Matrix([
[                                                                             -l*sin(theta_1(t)),                                                                             0,                                            0],
[                                                                              l*cos(theta_1(t)),                                                                             0,                                            0],
[                                            -l*(sin(theta_1(t) + theta_2(t)) + sin(theta_1(t))),                                               -l*sin(theta_1(t) + theta_2(t)),                                            0],
[                                             l*(cos(theta_1(t) + theta_2(t)) + cos(theta_1(t))),                                                l*cos(theta_1(t) + theta_2(t)),                                            0],
[-l*(sin(theta_1(t) + theta_2(t)) + sin(theta_1(t) + theta_2(t) + theta_3(t)) + sin(theta_1(t))

In [165]:
(m_1, m_2, m_3) = symbols('m_1 m_2 m_3' , real = True)

mass_xy = Matrix([[m_1, 0, 0, 0, 0, 0],
                  [0, m_1, 0, 0, 0, 0],
                  [0, 0, m_2, 0, 0, 0],
                  [0, 0, 0, m_2, 0, 0],
                  [0, 0, 0, 0, m_3, 0],
                  [0, 0, 0, 0, 0, m_3]])
mass_xy

Matrix([
[m_1,   0,   0,   0,   0,   0],
[  0, m_1,   0,   0,   0,   0],
[  0,   0, m_2,   0,   0,   0],
[  0,   0,   0, m_2,   0,   0],
[  0,   0,   0,   0, m_3,   0],
[  0,   0,   0,   0,   0, m_3]])

In [166]:
M_jacobian = simplify(J.T * mass_xy * J)
M_jacobian

Matrix([
[l**2*(m_1 + 2*m_2*cos(theta_2(t)) + 2*m_2 + 2*m_3*cos(theta_2(t) + theta_3(t)) + 2*m_3*cos(theta_2(t)) + 2*m_3*cos(theta_3(t)) + 3*m_3), l**2*(m_2*cos(theta_2(t)) + m_2 + m_3*cos(theta_2(t) + theta_3(t)) + m_3*cos(theta_2(t)) + 2*m_3*cos(theta_3(t)) + 2*m_3), l**2*m_3*(cos(theta_2(t) + theta_3(t)) + cos(theta_3(t)) + 1)],
[              l**2*(m_2*cos(theta_2(t)) + m_2 + m_3*cos(theta_2(t) + theta_3(t)) + m_3*cos(theta_2(t)) + 2*m_3*cos(theta_3(t)) + 2*m_3),                                                                                l**2*(m_2 + 2*m_3*cos(theta_3(t)) + 2*m_3),                                l**2*m_3*(cos(theta_3(t)) + 1)],
[                                                                          l**2*m_3*(cos(theta_2(t) + theta_3(t)) + cos(theta_3(t)) + 1),                                                                                            l**2*m_3*(cos(theta_3(t)) + 1),                                                      l**2*m_3]])

In [167]:
g = symbols('g')

FG_1 = J1.T * Matrix([[0], [-g]])
FG_1

Matrix([
[-g*l*cos(theta_1(t))],
[                   0],
[                   0]])

In [168]:
FG_2 = J2.T * Matrix([[0], [-g]])
FG_2

Matrix([
[-g*l*(cos(theta_1(t) + theta_2(t)) + cos(theta_1(t)))],
[                    -g*l*cos(theta_1(t) + theta_2(t))],
[                                                    0]])

In [169]:
FG_3 = J3.T * Matrix([[0], [-g]])
FG_3

Matrix([
[-g*l*(cos(theta_1(t) + theta_2(t)) + cos(theta_1(t) + theta_2(t) + theta_3(t)) + cos(theta_1(t)))],
[                  -g*l*(cos(theta_1(t) + theta_2(t)) + cos(theta_1(t) + theta_2(t) + theta_3(t)))],
[                                                   -g*l*cos(theta_1(t) + theta_2(t) + theta_3(t))]])

In [170]:
G = FG_1 + FG_2 + FG_3
G

Matrix([
[-g*l*(cos(theta_1(t) + theta_2(t)) + cos(theta_1(t))) - g*l*(cos(theta_1(t) + theta_2(t)) + cos(theta_1(t) + theta_2(t) + theta_3(t)) + cos(theta_1(t))) - g*l*cos(theta_1(t))],
[                                                            -g*l*(cos(theta_1(t) + theta_2(t)) + cos(theta_1(t) + theta_2(t) + theta_3(t))) - g*l*cos(theta_1(t) + theta_2(t))],
[                                                                                                                                -g*l*cos(theta_1(t) + theta_2(t) + theta_3(t))]])

In [171]:
M_jacobian_subs = M_jacobian.subs({theta_1: pi/2, theta_2: 0, theta_3: 0})
M_jacobian_subs

Matrix([
[l**2*(m_1 + 4*m_2 + 9*m_3), l**2*(2*m_2 + 6*m_3), 3*l**2*m_3],
[      l**2*(2*m_2 + 6*m_3),   l**2*(m_2 + 4*m_3), 2*l**2*m_3],
[                3*l**2*m_3,           2*l**2*m_3,   l**2*m_3]])

In [172]:
G_subs = G.subs({theta_1: pi/2, theta_2: 0, theta_3: 0})
G_subs

Matrix([
[0],
[0],
[0]])